# CLV

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mahdi-ebrahimi-per/AI/blob/main/8-%20CLV/CLV.ipynb)




In [1]:
import numpy as np
import pandas as pd
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import math

from persiantools.jdatetime import JalaliDate
import datetime
from dateutil.relativedelta import relativedelta


from sklearn.cluster import KMeans
from sklearn.preprocessing import scale
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# import seaborn as sns

from colorama import Fore

from six import StringIO #DM
from IPython.display import Image, display
import pydot


### Load Transaction CSV
    - اطلاعات مربوط به تراکنش های مربوط به ترمینال هاست

In [2]:
# Load Data Frame
Trans_df = pd.read_csv("TRANSACTIONS.csv", encoding="utf-8-sig")
Trans_df = Trans_df.iloc[:, :10] #ignore last 2 columns
# print(Trans_df.head())


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Trans_df.columns:
    count = Trans_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
Features_includes_NaN = 0
for f in list(Trans_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        Features_includes_NaN += 1

if NaN_include_flag:
    print(Fore.RED + f"DataFrame Includes {sum(list(Trans_df.isna().sum()))} NaN Values in {Features_includes_NaN} Features 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+---------------+--------------+
|    Feature    | Unique Value |
+---------------+--------------+
|  TERMINAL_NO  |      11      |
|    TRANDATE   |      6       |
|  RECENCYDATE  |      27      |
|  TRANNUMDAYS  |      22      |
|   FREQUENCY   |      36      |
|     AMOUNT    |      49      |
| TRAN_PURCHASE |      35      |
|  TRAN_CHARGE  |      14      |
|  TRAN_BALANCE |      1       |
|    TERM_POS   |      0       |
+---------------+--------------+
DataFrame Includes 50 NaN Values in 1 Features 👎🏼


| TERMINAL_NO | TRANDATE | RECENCYDATE | TRANNUMDAYS | FREQUENCY | AMOUNT | TC | TRAN_PURCHASE | TRAN_CHARGE | TRAN_BALANCE |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| شماره ترمینال | تاریخ | آخرین خرید | تعداد روز هایی که در ماه تراکنش داشته | تعداد تراکنش | مبلغ کل تراکنش | کارمزد | تراکنش خرید | تراکنش شارژ | تراکنش دریافت موجودی |

### Load Merchant CSV
    -  ها است PSP این دیتاست مربوط به اطلاعات  

In [3]:
# Load Data Frame
Merch_df = pd.read_excel("MERCHANTS.xlsx")
# print(df.head())


# Unique Value
unique_counts = PrettyTable()
unique_counts.field_names = ["Feature", "Unique Value"]

for column in Merch_df.columns:
    count = Merch_df[column].nunique()
    unique_counts.add_row([column, count])


print(unique_counts)    


# NaN Values
NaN_include_flag = False
Features_includes_NaN = 0
for f in list(Merch_df.isna().sum()):
    if f > 0:
        NaN_include_flag = True
        Features_includes_NaN += 1

if NaN_include_flag:
    print(Fore.RED + f"DataFrame Includes {sum(list(Trans_df.isna().sum()))} NaN Values in {Features_includes_NaN} Features 👎🏼" + Fore.RESET)

else:
    print(Fore.GREEN + "DataFrame Haven't any NaN Values 👍🏼" + Fore.RESET)


+-----------------+--------------+
|     Feature     | Unique Value |
+-----------------+--------------+
|   TERMINAL_NO   |      11      |
| MAININSTALLDATE |      8       |
|  DISABLED_DATE  |      0       |
|  CATEGORY_NAME  |      10      |
|   CATEGORYCODE  |      10      |
|   PROVINCENAME  |      7       |
|   PROVINCECODE  |      7       |
|     CITYNAME    |      9       |
|     CITYCODE    |      9       |
|   PROJECTNAME   |      2       |
|   PROJECTCODE   |      2       |
|    AGENTNAME    |      7       |
|    AGENTCODE    |      7       |
|  TERMINALSTATUS |      2       |
|     POSTYPE     |      2       |
|     POSMODEL    |      3       |
|     POSBRAND    |      3       |
| WORKFLOWCAPTION |      2       |
|       TEL1      |      11      |
|   FIRSTNAMEFA   |      11      |
|    LASTNAMEFA   |      11      |
|  OFFICEADDRESS  |      11      |
|      MOBILE     |      11      |
+-----------------+--------------+
DataFrame Includes 50 NaN Values in 5 Features 👎🏼


### Join two DataFrame according to "Merchant"

In [4]:

DataFrame = pd.concat([Merch_df, Trans_df], axis=1).reindex(Merch_df.index)


print(DataFrame.isna().sum())




TERMINAL_NO         0
MAININSTALLDATE     1
DISABLED_DATE      11
CATEGORY_NAME       0
CATEGORYCODE        0
PROVINCENAME        0
PROVINCECODE        0
CITYNAME            0
CITYCODE            0
PROJECTNAME         0
PROJECTCODE         0
AGENTNAME           0
AGENTCODE           0
TERMINALSTATUS      0
POSTYPE             1
POSMODEL            1
POSBRAND            1
WORKFLOWCAPTION     0
TEL1                0
FIRSTNAMEFA         0
LASTNAMEFA          0
OFFICEADDRESS       0
MOBILE              0
TERMINAL_NO         0
TRANDATE            0
RECENCYDATE         0
TRANNUMDAYS         0
FREQUENCY           0
AMOUNT              0
TRAN_PURCHASE       0
TRAN_CHARGE         0
TRAN_BALANCE        0
TERM_POS           11
dtype: int64


### Seprate Date into two category 
    - two category with same len

In [5]:


def seprate_time(DataFrame,TimeStepMonth_Seperator=3 ,time_ColumnName="TRANDATE" ,index_ColumnName="TERMINAL_NO"):
    
    Time_df = DataFrame[time_ColumnName]
    
    
    # convert Jalalian to Gregorian
    df_jalaliToGregorian = []
    df_TerminalNumber = []
    
    # print(DataFrame)
    
    for index, row in DataFrame.iterrows():
        # print(int(row["TRANDATE"]))
        gregorian_date = JalaliDate(int(str(int(row["TRANDATE"]))[:4]), int(str(int(row["TRANDATE"]))[4:]),1).to_gregorian()
        df_TerminalNumber.append(int(row["TERMINAL_NO"]))
        df_jalaliToGregorian.append(gregorian_date)


    # Delete ME
    df_jalaliToGregorian.append(datetime.date(2000,10,1))
    df_jalaliToGregorian.append(datetime.date(2030,10,30))

    # Find Min and Max of Dates    
    date_min_max = pd.DataFrame(df_jalaliToGregorian).agg(["min", "max"])

    
    def time_plus_month(GivenDate, Months=TimeStepMonth_Seperator):
        date_format = '%Y/%m/%d'
        dtObj = datetime.datetime.strptime(GivenDate, date_format)
        future_date = dtObj + relativedelta(months=Months)
        return future_date

    middle = time_plus_month(str(date_min_max[0]['min']).replace("-","/"))
    
    Time_Category_Upper = []
    Time_Category_Under = [] 
    
    
    for date in df_jalaliToGregorian :
        if date > middle.date():
            Time_Category_Upper.append(date)
        else:
            Time_Category_Under.append(date)
    
    # print(Time_Category_Upper) 
    # print(Time_Category_Under)
    
    
    
print(seprate_time(Trans_df, 4))


None


### Make Unique Terminal number per sumOfFeatures For First Category of dates (1 to 3 month)
    - use maximum for RECENCYDATE

In [6]:
Trans_df.head()


,TERMINAL_NO,TRANDATE,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE,TERM_POS
0,410001,140012,14001225,1,3,150000,0,3,0,NaN
1,410001,140102,14010201,1,2,60000,0,2,0,NaN
2,410025,140103,14010331,22,58,83250000,58,0,0,NaN
3,410025,140012,14001227,12,24,57900000,24,0,0,NaN
4,410025,140104,14010404,3,8,12950000,8,0,0,NaN


In [7]:
TerminalNo_Unique = list(Trans_df.TERMINAL_NO.unique())


# Columns = ["TERMINAL_NO"," RECENCYDATE", "TRANNUMDAYS", "FREQUENCY", "AMOUNT", "TC", "TRAN_PURCHASE", "TRAN_CHARGE", 'TRAN_BALANCE']
Columns = ["TERMINAL_NO"," RECENCYDATE", "TRANNUMDAYS", "FREQUENCY", "AMOUNT", "TRAN_PURCHASE", "TRAN_CHARGE", 'TRAN_BALANCE']


Max_RECENCYDATE = []
Sum_TRANNUMDAYS = []
Sum_FREQUENCY = []
Sum_AMOUNT = []
# Sum_TC = []
Sum_TRAN_PURCHASE = []
Sum_TRAN_CHARGE = []
Sum_TRAN_BALANCE = []

# Max_RECENCYDATE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["RECENCYDATE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Max_RECENCYDATE.append(max(Listed))

# print(Max_RECENCYDATE)

# Sum_TRANNUMDAYS
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRANNUMDAYS"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRANNUMDAYS.append(sum(Listed))


# Sum_FREQUENCY
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["FREQUENCY"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_FREQUENCY.append(sum(Listed))



# Sum_AMOUNT
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["AMOUNT"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_AMOUNT.append(sum(Listed))



# # Sum_TC
# for Terminal in TerminalNo_Unique:
#     Listed = Trans_df["TC"][(Trans_df["TERMINAL_NO"]==Terminal)]
#     Sum_TC.append(sum(Listed))



# Sum_TRAN_PURCHASE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_PURCHASE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_PURCHASE.append(sum(Listed))



# Sum_TRAN_CHARGE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_CHARGE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_CHARGE.append(sum(Listed))


# Sum_TRAN_BALANCE
for Terminal in TerminalNo_Unique:
    Listed = Trans_df["TRAN_BALANCE"][(Trans_df["TERMINAL_NO"]==Terminal)]
    Sum_TRAN_BALANCE.append(sum(Listed))


# zipped = list(zip(TerminalNo_Unique, Max_RECENCYDATE, Sum_TRANNUMDAYS, Sum_FREQUENCY, Sum_AMOUNT, Sum_TC,
#                   Sum_TRAN_PURCHASE, Sum_TRAN_CHARGE, Sum_TRAN_BALANCE))

zipped = list(zip(TerminalNo_Unique, Max_RECENCYDATE, Sum_TRANNUMDAYS, Sum_FREQUENCY, Sum_AMOUNT,
                  Sum_TRAN_PURCHASE, Sum_TRAN_CHARGE, Sum_TRAN_BALANCE))




DataFrame = pd.DataFrame(zipped, columns=Columns)

DataFrame





,TERMINAL_NO,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE
0,410001,14010201,2,5,210000,0,5,0
1,410025,14010404,74,172,1299800000,172,0,0
2,410027,14010404,30,61,1992040000,59,2,0
3,410028,14010404,47,158,1309442000,155,3,0
4,410029,14010404,55,105,3705047000,21,84,0
5,410033,14001123,12,48,74450000,48,0,0
6,410035,14010404,114,1949,552105106,1937,12,0
7,410036,14010324,27,41,979920000,28,13,0
8,410038,14010331,38,53,48375000,53,0,0
9,410050,14010404,140,3110,1424296000,3068,42,0


### Cluster Terminals Based on (AMOUNT or TC) For Secend Category of dates (4 to 6 month)
    

In [8]:
TerminalNo_Unique = list(Trans_df.TERMINAL_NO.unique())

model = KMeans(n_clusters=3)
model = model.fit(scale(Sum_AMOUNT).reshape(-1,1))
# print(model.labels_)
idx = np.argsort(model.cluster_centers_.sum(axis=1))
lut = np.zeros_like(idx)
lut[idx] = np.arange(3)


Labels = {"CLUSTER" : np.array(lut[model.labels_])}
Labels_DataFrame = pd.DataFrame(Labels)
# print(Labels)



uniqueTerminal_Features_Merchant = pd.merge(DataFrame, Merch_df, on="TERMINAL_NO")


uniqueTerminal_Features_Merchant_Cluster = pd.concat([uniqueTerminal_Features_Merchant, pd.DataFrame(Labels)], axis=1)

DataFrame = uniqueTerminal_Features_Merchant_Cluster

DataFrame.head()


c:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


,TERMINAL_NO,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE,MAININSTALLDATE,DISABLED_DATE,...,POSTYPE,POSMODEL,POSBRAND,WORKFLOWCAPTION,TEL1,FIRSTNAMEFA,LASTNAMEFA,OFFICEADDRESS,MOBILE,CLUSTER
0,410001,14010201,2,5,210000,0,5,0,NaN,NaN,...,NaN,NaN,NaN,تکميل مدارک,2189710001,رايان ارتباط شب آهنگ,رايان ارتباط شب آهنگ,ميدان ونک خيابان شيرازي شمالي خيابان گرمسار غر...,9027890061,0
1,410025,14010404,74,172,1299800000,172,0,0,1399/07/28,NaN,...,LanDialup,V5,Castles,تکميل مدارک,4533516228,سيدنورالدين,ميرجعفريان,اردبيل آبي بيگلو خيابان سبلان کوچه جعفريان پ ...,9143537600,1
2,410027,14010404,30,61,1992040000,59,2,0,1399/07/24,NaN,...,LanDialup,V5,Castles,تکميل مدارک,3442233090,عليرضا,کارآموز,سيرجان بلوار دکتر صادقي حدفاصل چهارراه موحدي و...,9131456320,1
3,410028,14010404,47,158,1309442000,155,3,0,1400/03/13,NaN,...,GPRS,G3,Nexgo,تکميل مدارک,3532354110,حمل ونقل آذرخش بار ميبد,حمل ونقل آذرخش بار ميبد,ميبد جاده سنتو جنب ايران خودرو,9196712006,1
4,410029,14010404,55,105,3705047000,21,84,0,1399/07/19,NaN,...,LanDialup,V5,Castles,تکميل مدارک,4533333749,يوسف,اوجاققلي زاده,اردبيل کارشناسان فاز2 خيابان وصال کوچه شهباززا...,9141563187,2


In [9]:
print(DataFrame["DISABLED_DATE"].unique())

[nan]


### Feature Engineering
    - Delete Column (Feature) that we don't need it
        - It's better to use Code instead name, eg : CityCode instead CityName
    - Make Better New Features From Old Features

    - is it True :
        - Tel1 : Drop
        - Use len Office address as attribute (Office on the right)
        - fillna with 0

In [10]:

# MAININSTALLDATE
# Use Just year, eg : 1399/03/01 -> 1399 | NaN -> 0
DataFrame["MAININSTALLDATE"] = DataFrame["MAININSTALLDATE"].fillna(0)
DataFrame["MAININSTALLDATE"] = [int(str(item)[:4]) for item in DataFrame["MAININSTALLDATE"]]


# OFFICEADDRESS
# Use len insteal full address as attribute
DataFrame["MAININSTALLDATE"] = DataFrame["MAININSTALLDATE"].fillna(0)
DataFrame["OFFICEADDRESS"] = [len(str(item)) for item in DataFrame["OFFICEADDRESS"]]


# Drop Columns
HaveToDrop = ["DISABLED_DATE", "CATEGORY_NAME", "PROVINCENAME",
                "CITYNAME", "PROJECTNAME", "AGENTNAME", "TEL1",
                "FIRSTNAMEFA", "LASTNAMEFA", "TERMINAL_NO"]

DataFrame = DataFrame.drop(columns=HaveToDrop)


# MOBILE
# Find Operator and replace Mobile Number With Operator name, and then oneHot it!
def MobileOperator():
    DataFrame["MOBILE"] = DataFrame["MOBILE"].fillna(0)
    DataFrame["MOBILE"] = [int(str(item)[:3]) for item in DataFrame["MOBILE"]]

    mci = [912, 990, 919, 910, 911, 918, 917, 916, 915, 914, 913]
    irancell = [935, 936, 937, 938, 939, 901, 902, 903, 904, 905]

    index = 0
    for number in DataFrame["MOBILE"]:
        if number in mci:
            DataFrame["MOBILE"][index] = "mci"
            index += 1
        
        elif number in irancell:
            DataFrame["MOBILE"][index] = "irancell"
            index += 1
        
        else:
            DataFrame["MOBILE"][index] = "Another"
            index += 1
            
MobileOperator()
    

DataFrame.head()

<ipython-input-10-0db03519e317>:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["MOBILE"][index] = "irancell"
c:\Users\asus\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-10-0db03519e317>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataFrame["MOBILE"][index] = "mci"


,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE,MAININSTALLDATE,CATEGORYCODE,PROVINCECODE,...,PROJECTCODE,AGENTCODE,TERMINALSTATUS,POSTYPE,POSMODEL,POSBRAND,WORKFLOWCAPTION,OFFICEADDRESS,MOBILE,CLUSTER
0,14010201,2,5,210000,0,5,0,0,4812,1,...,5050,124,فعال,NaN,NaN,NaN,تکميل مدارک,63,irancell,0
1,14010404,74,172,1299800000,172,0,0,1399,5051,7,...,1010,85,فعال,LanDialup,V5,Castles,تکميل مدارک,59,mci,1
2,14010404,30,61,1992040000,59,2,0,1399,5722,9,...,1010,99,فعال,LanDialup,V5,Castles,تکميل مدارک,51,mci,1
3,14010404,47,158,1309442000,155,3,0,1400,4214,10,...,1010,106,فعال,GPRS,G3,Nexgo,تکميل مدارک,30,mci,1
4,14010404,55,105,3705047000,21,84,0,1399,7278,7,...,1010,85,فعال,LanDialup,V5,Castles,تکميل مدارک,57,mci,2


### one hot encoding   
    - Use one hot endcoding for string categorized columns
        for example : column have two unique value men and women
        we use :
            10 for men
            01 for women
            00 for NaN 

In [11]:

def oneHot_Column(DataFrame, ColumnName, inplace=False):
    uniqueValues = list(DataFrame[ColumnName].unique())
    uniqueValues_withOutNaN = []
        
    
    # uniqueValues without nan
    index = 0
    for uni in uniqueValues:
        try :
            math.isnan(uni)
            
            continue
            
        except:
            pass
        
        uniqueValues_withOutNaN.append(uni)
        
    
        
    Transformed = []
    
    # add full zero if nan value exist
    if len(uniqueValues) != len(uniqueValues_withOutNaN):
        Transformed.append("0"*len(uniqueValues_withOutNaN))
    
    
    
    index = 0
    for uniq in uniqueValues_withOutNaN:
        BasicZeros = [0] * len(uniqueValues_withOutNaN)
        BasicZeros[index] = 1
        
        Transformed.append("".join(map(str,BasicZeros)))
        index += 1
        
     
    unique_oneHot = dict(zip(uniqueValues, Transformed)) # dictionary of {each unique value : OneHot code} 


    # Replece in column    
    if inplace == True:
        for i in unique_oneHot.items():
            DataFrame[ColumnName].replace(i[0], int(i[1]), inplace=True)
            
    else:
        newDataFrame = DataFrame.copy()
        for i in unique_oneHot.items():
            newDataFrame[ColumnName] = newDataFrame[ColumnName].replace(i[0], int(i[1])) # Delete me when inplace is True
    
        return newDataFrame
    
            
    


In [12]:
needToOneHot = ["TERMINALSTATUS", "POSTYPE","POSMODEL", "POSBRAND", "WORKFLOWCAPTION", "MOBILE"]

for column in needToOneHot:
    oneHot_Column(DataFrame, column, inplace=True)



DataFrame.head()


,RECENCYDATE,TRANNUMDAYS,FREQUENCY,AMOUNT,TRAN_PURCHASE,TRAN_CHARGE,TRAN_BALANCE,MAININSTALLDATE,CATEGORYCODE,PROVINCECODE,...,PROJECTCODE,AGENTCODE,TERMINALSTATUS,POSTYPE,POSMODEL,POSBRAND,WORKFLOWCAPTION,OFFICEADDRESS,MOBILE,CLUSTER
0,14010201,2,5,210000,0,5,0,0,4812,1,...,5050,124,10,0,0,0,10,63,10,0
1,14010404,74,172,1299800000,172,0,0,1399,5051,7,...,1010,85,10,10,100,100,10,59,1,1
2,14010404,30,61,1992040000,59,2,0,1399,5722,9,...,1010,99,10,10,100,100,10,51,1,1
3,14010404,47,158,1309442000,155,3,0,1400,4214,10,...,1010,106,10,1,10,10,10,30,1,1
4,14010404,55,105,3705047000,21,84,0,1399,7278,7,...,1010,85,10,10,100,100,10,57,1,2


### Finally Check DataFrame
    - Check DataFrame not include any string, because of Decision Tree 

In [13]:
# Check Final Feed DataFrame has string or not

flag = False
errors = []

for column in DataFrame.columns:
    if any( [type(item) == str for item in DataFrame[column]]):
        flag = True
        errors.append(column,":" ,DataFrame[column][0],"|", DataFrame[column][1])

    
if flag:
    print(Fore.RED, "Final DataFrame includes String object 👎🏼, Strings Are : ")
    for err in errors:
        print(err)
        
    print(Fore.RESET)

else:
    print(Fore.GREEN, "Final DataFrame is clean of String object 👍🏼", Fore.RESET)




 Final DataFrame is clean of String object 👍🏼 


### Fit Decision Tree

In [14]:
Train_Features  = list(DataFrame.columns[:31])

y = DataFrame["CLUSTER"]
x = DataFrame[Train_Features]

clf = tree.DecisionTreeClassifier()
clf.fit(x, y)



DecisionTreeClassifier()

### New Data Prediction


In [15]:
clf = RandomForestClassifier(n_estimators=10)

clf = clf.fit(x, y)

# values = [[10, 1, 4, 0, 0, 0]]
prediction = clf.predict(values)  # [1] OR [0]
print( f"{values[0]}\n","Hired:", bool(prediction))

ValueError: X has 6 features, but DecisionTreeClassifier is expecting 21 features as input.

### Decesion Tree Graph 

In [ ]:
# dot_data = StringIO()
# tree.export_graphviz(clf, out_file=dot_data, feature_names=Train_Features)
# graph = pydot.graph_from_dot_data(dot_data.getvalue())

# plt = Image(pydot.create_png())
# display.svg(plt)